# Naive Bayes on the Emoji data

In [132]:
import random
import twitter
import emoji
# import itertools
import pandas as pd
from itertools import chain
import numpy as np
#from sklearn.model_selection import train_test_split
import os
# import nb_utils


In [133]:
all_tweets = pd.read_csv('data/emojis_homemade.csv')
all_tweets['emoji'].value_counts()

/home/nickdbn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


😂        54429
😭        22356
❤        18764
emoji    12865
😍        12327
🔥        12058
🤣         7587
🙏         6189
😩         6102
🤔         5914
💕         5878
😊         5349
🖤         5252
✨         4781
👀         4674
💜         4648
👏         4383
🙄         4221
💀         4188
🎉         4030
🙌         3998
😘         3835
💯         3572
💙         3413
😔         3272
😏         3141
👍         3137
♥         3092
💖         2936
😉         2910
         ...  
🉐            1
🔚            1
🕕            1
🗜            1
◀            1
🧘            1
👝            1
👲            1
㊗            1
🏣            1
👷            1
🤵            1
🚛            1
🀄            1
🚟            1
🈳            1
🈲            1
🗃            1
🥫            1
🛤            1
🕍            1
🏬            1
➿            1
🔕            1
🚈            1
🕟            1
📗            1
📇            1
📤            1
🤾            1
Name: emoji, Length: 1077, dtype: int64

In [134]:
all_tweets.shape

(605813, 3)

In [135]:
tweets = all_tweets.groupby('emoji').filter(lambda c:len(c) > 500)
print ("tweets shape:", tweets.shape)
tweets['emoji'].value_counts()

tweets shape: (380208, 3)


😂        54429
😭        22356
❤        18764
emoji    12865
😍        12327
🔥        12058
🤣         7587
🙏         6189
😩         6102
🤔         5914
💕         5878
😊         5349
🖤         5252
✨         4781
👀         4674
💜         4648
👏         4383
🙄         4221
💀         4188
🎉         4030
🙌         3998
😘         3835
💯         3572
💙         3413
😔         3272
😏         3141
👍         3137
♥         3092
💖         2936
😉         2910
         ...  
👋          723
🤘          722
🌎          715
™          706
😃          705
🎥          693
🚀          680
📺          637
💰          605
☀          600
📹          590
💃          589
😝          584
🤭          575
®          562
🤨          554
😻          552
😓          549
🤢          547
😛          547
🧐          545
🤙          538
🎂          533
⚽          532
📽          529
🍞          519
🐐          517
☠          511
😮          510
🎧          506
Name: emoji, Length: 144, dtype: int64

In [136]:
max(tweets['text'], key=lambda t:len(t))

'RT @JoanKareri: A guy who smells good and leaves you with his scent after he leaves&gt;&gt;&gt;&gt; &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;'

# Text processing
Let's try filtering out twitter handles (any words starting with '@')
We might consider doing the same to hashtags

In [137]:
f = lambda txt : ' '.join(word for word in txt.split(' ') if not word.startswith('@'))

tweets['text'] = tweets['text'].apply(f)

/home/nickdbn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [138]:
tweets['text'][0]

'RT [VID] 181023 - Foi adicionada a letra “D” no outdoor misterioso do #BTS em Hollywood.\nFormando: BTS AND... \n\n ILOVEPAR…'

In [139]:
chars = list(sorted(set(chain(*tweets['text']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in tweets['text'])

emojis = list(sorted(set(tweets['emoji'])))
emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
emojis[:10]

['emoji', '©', '®', '‼', '™', '▶', '☀', '☠', '☹', '☺']

In [140]:
#train_tweets = tweets[0:10000]
#dev_tweets = tweets[10000:20000]
#test_tweets = tweets[20000:30000]


# !! Temp! Try on much bigger dataset
#train_tweets = tweets[0:100000] # 100000 tweets
#dev_tweets = tweets[100000:110000]
#test_tweets = tweets[110000:120000]

# All the data
train_tweets = tweets[0:-20000] # 100000 tweets
dev_tweets = tweets[-20000:-10000]
test_tweets = tweets[-10000:-1]


In [141]:
import sklearn
import numpy as np
import collections 

In [142]:
from sklearn import feature_extraction, preprocessing, naive_bayes, metrics

In [143]:
vocab_size = 5000

#tfidf_vec = feature_extraction.text.TfidfVectorizer(max_features=vocab_size)
tfidf_vec = feature_extraction.text.TfidfVectorizer(max_features=vocab_size,
                                                   use_idf = False) # try without idf weighting
                                                   #min_df = 50) # minimum occurrances, trying to filter out @handles
label_encoder = preprocessing.LabelEncoder()
train_x = tfidf_vec.fit_transform(train_tweets['text'])
train_y = label_encoder.fit_transform(train_tweets['emoji'])
dev_x = tfidf_vec.fit_transform(dev_tweets['text'])
dev_y = label_encoder.fit_transform(dev_tweets['emoji'])

In [144]:
bayes = naive_bayes.MultinomialNB()
bayes.fit (train_x, train_y)
pred = bayes.predict (train_x) # dev_x??
metrics.precision_score(pred, train_y, average='micro') # dev_y?

0.3258478434682184

In [145]:
d = np.eye (len(tfidf_vec.vocabulary_))
word_pred = bayes.predict_proba(d)

In [146]:
tfidf_vec.vocabulary_

{'amp': 335,
 'follow': 1623,
 'back': 511,
 'https': 2087,
 'co': 1034,
 'rt': 3621,
 'the': 4238,
 'mc': 2715,
 'asked': 438,
 'what': 4648,
 'is': 2259,
 'secret': 3720,
 'of': 3043,
 'junmyeon': 2376,
 'popularity': 3285,
 'be': 578,
 'honest': 2052,
 'my': 2899,
 'ho': 2033,
 'pastor': 3173,
 'said': 3645,
 'you': 4830,
 'can': 855,
 'do': 1259,
 'right': 3585,
 'thing': 4254,
 'wrong': 4751,
 'way': 4616,
 'and': 339,
 'lose': 2603,
 'your': 4833,
 'blessing': 670,
 'let': 2538,
 'that': 4236,
 'sink': 3869,
 'in': 2180,
 'best': 614,
 'luck': 2622,
 'team': 4202,
 'sarkar': 3670,
 'sarkardiwali': 3671,
 'thalapathy': 4227,
 'wishes': 4694,
 'from': 1676,
 'watchm': 4612,
 'popular': 3284,
 'leftist': 2527,
 'website': 4625,
 'urges': 4471,
 'readers': 3475,
 'to': 4296,
 'cut': 1142,
 'through': 4274,
 'border': 715,
 'assist': 447,
 'migrant': 2776,
 'votered': 4555,
 'im': 2163,
 'bitch': 656,
 'act': 220,
 'funny': 1694,
 'cause': 897,
 'when': 4651,
 'it': 2266,
 'turn': 437

In [147]:
# is this the missing line of code??

inverse_vocab = {idx: word for word, idx in tfidf_vec.vocabulary_.items()}

In [148]:
for i in range (10):
    print (inverse_vocab[i])

00
000
00pm
01
011
0292k
05
06
07ue1sykzq
08


In [149]:
from collections import Counter, defaultdict
by_cls = defaultdict(Counter)
for word_idx, pred in enumerate(word_pred):
    for class_idx, score in enumerate(pred):
        cls = label_encoder.classes_[class_idx]
        by_cls[cls][inverse_vocab[word_idx]] = score

In [150]:
for k in by_cls:
    words = [x[0] for x in by_cls[k].most_common(10)]
    print (k, ':', ' '.join(words))

emoji : tomorrow 5th 011 5sos 6pm 7pm 9k 0292k rate 5m
© : 15 16 𝙞𝙛 ptvtf1bszw 𝘴𝘵𝘢𝘳𝘴 137 14 150 𝚝𝚒𝚖𝚎 13
® : qxqik3u5hh post other endless born btob neave catch thai volume
‼ : bruno_the_british_shorthair blowing damm uber ol nfl bounce tomarrow huh makeup
™ : first apply successfully qxqik3u5hh alright post born although la men
▶ : 1tpkau0wld nowadays 602 name brothers c32wbfza8h goosebumps kita joking chai
☀ : ka tagging psmxxcnlrg take taehyung tae murdered 123 grades touched
☠ : yessssshaesunglee miss snuck flirt arrest display mr hiddleston shownu essay
☹ : walk secret greatly several unbothered says sadly iheartawards monkey thomas
☺ : 𝘀𝗸𝘆 characters bonus nigeria sree 0xi8mvcyyt own humbstle lang 비투비
♥ : 9unqu8sbs7 hashtag makers 1t 1pm reviews hahahah ako richard al
⚠ : wishing every s2qzmpa67m alabama greet shares feet weather angels medals
⚡ : email laid fortnite present angels kissing ain announced game decided
⚽ : american science matt soundcloud laughing moma caught why can

In [151]:
by_cls['🤯'].most_common(10) # mind blown emoji

[('champions', 0.27732145203296277),
 ('mama2018', 0.27732145203296277),
 ('chapter', 0.2498184467405464),
 ('braces', 0.0749430257091778),
 ('their', 0.07293684244477991),
 ('haven', 0.0518339117024928),
 ('bqgjsgsk9z', 0.036099173077651654),
 ('wmxfusuvxf', 0.03608927302104026),
 ('woahhhh', 0.031899074785133555),
 ('ministerial', 0.02676208009202716)]